<a href="https://colab.research.google.com/github/vvikasreddy/lexically_constrained_beam_search_/blob/main/beam_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## downloading essential modules

In [45]:
!pip install datasets

In [ ]:
# Using the cs-en translation

In [46]:
from datasets import load_dataset

import torch
from transformers import MarianMTModel, MarianTokenizer



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [47]:
ds = load_dataset("wmt/wmt16", "tr-en")

In [48]:
ds

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 205756
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1001
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
})

In [49]:
ds['train'][0]

{'translation': {'en': "Kosovo's privatisation process is under scrutiny",
  'tr': "Kosova'nın özelleştirme süreci büyüteç altında"}}

In [50]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tr-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-tr-en")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/840k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [51]:
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62389, 512, padding_idx=62388)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62389, 512, padding_idx=62388)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [52]:
# https://huggingface.co/docs/transformers/model_doc/marian

In [53]:
def generate_translation(src_text, max_length = 50):

  # Tokenize input
  encoder_inputs = tokenizer(src_text, return_tensors="pt")

  # intializes the decoder input with decoder start token
  decoder_input = torch.tensor([[model.config.decoder_start_token_id]])

  # change the model to eval mode.
  model.eval()
  with torch.no_grad():

    generated_tokens = []

    while len(generated_tokens) < max_length:

      outputs = model(
          input_ids=encoder_inputs.input_ids,
          attention_mask=encoder_inputs.attention_mask,
          decoder_input_ids=decoder_input
      )


      next_token_logits = outputs.logits[:, -1, :]


      # get the token with maximum logits value
      next_token = torch.argmax(next_token_logits, dim=-1)

      generated_tokens.append(next_token.item())

      if next_token.item() == tokenizer.eos_token_id:
          break
      print(decoder_input)
      decoder_input = torch.cat([decoder_input, next_token.unsqueeze(0)], dim=1)
      print(decoder_input)
    translated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
  return translated_text


In [ ]:

### print("\nStep by step generation:")
# for i in range(len(generated_tokens)):
#     partial_text = tokenizer.decode(generated_tokens[:i+1], skip_special_tokens=True)
#     print(f"Step {i+1}: {partial_text}")

In [ ]:
model.config.num_beams

4

In [54]:
print(generate_translation(ds['validation'][1]['translation']['tr']))

tensor([[62388]])
tensor([[62388, 31058]])
tensor([[62388, 31058]])
tensor([[62388, 31058,     4]])
tensor([[62388, 31058,     4]])
tensor([[62388, 31058,     4,     8]])
tensor([[62388, 31058,     4,     8]])
tensor([[62388, 31058,     4,     8,  1129]])
tensor([[62388, 31058,     4,     8,  1129]])
tensor([[62388, 31058,     4,     8,  1129,  1782]])
tensor([[62388, 31058,     4,     8,  1129,  1782]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206,  3094]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206,  3094]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206,  3094,     9]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206,  3094,     9]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206,  3094,     9, 11363]])
tensor([[62388, 31058,     4,     8,  1129,  1782,   206,  3094,     9, 11363]])
tensor([[62388, 31058,     4

In [55]:
ds["validation"][1]["translation"]["tr"]

"Norveç'in beş milyon insanı en yüksek yaşam standartlarının tadını çıkarıyor, sadece Avrupa'da değil, dünyada."

In [56]:
ds["validation"][1]["translation"]["en"]

"Norway's five million people enjoy one of the highest standards of living, not just in Europe, but in the world."

In [29]:
# reference to get the python file from github



fatal: destination path 'lexically_constrained_beam_search_' already exists and is not an empty directory.


https://colab.research.google.com/github/jckantor/cbe61622/blob/master/docs/A.02-Downloading_Python_source_files_from_github.ipynb

In [37]:
# get constraints

user = "vvikasreddy"
repo = "lexically_constrained_beam_search_"
pyfile = "constraints.py"

# url = "https://github.com/vvikasreddy/lexically_constrained_beam_search_/blob/main/constraints.py"
url = f"https://raw.githubusercontent.com/{user}/{repo}/main/{pyfile}"
!wget --no-cache --backups=1 {url}

import constraints

--2024-11-02 19:54:00--  https://raw.githubusercontent.com/vvikasreddy/lexically_constrained_beam_search_/main/constraints.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4406 (4.3K) [text/plain]
Failed to rename constraints.py to constraints.py.1: (2) No such file or directory
Saving to: ‘constraints.py’

constraints.py      100%[===================>]   4.30K  --.-KB/s    in 0s      

2024-11-02 19:54:00 (41.5 MB/s) - ‘constraints.py’ saved [4406/4406]



In [42]:
# takes nice 5 minutes to get all the constraints
c = constraints.get_constraints()

KeyboardInterrupt: 

In [149]:
def generate_translation(src_text, decoder_input = [],  max_length = 1):

  """generate the decoder input ids and corresponding probabilities"""


  # Tokenize input
  encoder_inputs = tokenizer(src_text, return_tensors="pt")


  # initialize if not initialized.
  if not decoder_input:
    # because the intial decoder start token has probabilitty 1
    probabilities = torch.tensor([[1]])
    decoder_input = torch.tensor([[model.config.decoder_start_token_id]])

  # change the model to eval mode.
  model.eval()
  with torch.no_grad():

    generated_tokens = []

    outputs = model(
        input_ids=encoder_inputs.input_ids,
        attention_mask=encoder_inputs.attention_mask,
        decoder_input_ids=decoder_input
    )

    next_token_logits = outputs.logits[:, -1, :]

    next_token = torch.argmax(next_token_logits, dim=-1)


    top_probs, top_indices = torch.topk(torch.softmax(next_token_logits, dim=-1), k = 5)


  decoder_iputs = []
  probs = []
  for indx, id in enumerate(top_indices[0]):
    # print(decoder_input, id.unsqueeze(0).unsqueeze(0), id, "jhere")
    # print(probabilities, top_probs[0][indx].unsqueeze(0).unsqueeze(0))
    decoder_iputs.append(torch.cat([decoder_input, id.unsqueeze(0).unsqueeze(0)], dim=1))
    probs.append(torch.cat([probabilities, top_probs[0][indx].unsqueeze(0).unsqueeze(0)], dim=1))

  # print(decoder_iputs, probs)
  return decoder_iputs, probs
  # return decoder_input, probabilities


In [148]:
print(generate_translation(ds['validation'][1]['translation']['tr']))

[tensor([[62388, 31058]]), tensor([[62388,  3884]]), tensor([[62388,  1014]]), tensor([[62388,    71]]), tensor([[62388, 36600]])] [tensor([[1.0000, 0.4112]]), tensor([[1.0000, 0.1607]]), tensor([[1.0000, 0.0560]]), tensor([[1.0000, 0.0387]]), tensor([[1.0000, 0.0166]])]
([tensor([[62388, 31058]]), tensor([[62388,  3884]]), tensor([[62388,  1014]]), tensor([[62388,    71]]), tensor([[62388, 36600]])], [tensor([[1.0000, 0.4112]]), tensor([[1.0000, 0.1607]]), tensor([[1.0000, 0.0560]]), tensor([[1.0000, 0.0387]]), tensor([[1.0000, 0.0166]])])


In [44]:
def beam_search(maxlen, numC, k, src):

    grids = [[[0 for _ in range(k)] for _ in range(numC)] for _ in range(maxlen)]

    # intialize the first grid to start hyp
    for i in range(k):
        grids[0][0][i] = 'start'

    print(grids)
    grids[0][0]

    for t in range(1, maxlen):

        index_c = max(0, (numC - t) - maxlen)

        for c in range(index_c, min(t, numC) + 1):

            n = []
            s = []
            g = []

            for element in grids[t-1][c]:
                if element == 'start':
                  g = generate_translation(src_text= src, decoder_input = [], max_len = 4)

            if c > 0:
              for element in grids[t-1][c-1]:
                if element == 'start':
                  g = generate_translation(src_text= src, decoder_input = [], max_len = 4)


            break
        break

    # sanity : print grids
    print(grids)


beam_search(maxlen= 4, numC=2, k = 5)

[[['start', 'start', 'start', 'start', 'start'], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]]
start
start
start
start
start
[[['start', 'start', 'start', 'start', 'start'], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]]
